# VotecounterTest
A script for setting up and executing performance tests on votecounters.

## Dependencies

In [1]:
import json
import time
from VoteCount import VoteCount
import VoteCounter
from tqdm import tqdm
import string
from copy import deepcopy

# to help generate formatted output rnepresentations
from IPython.core.display import display, HTML
import markdown2 as md

def html(markdown_string):
    display(HTML(md.markdown(markdown_string)))
    
# for simplifying text to just numbers
no_punctuation = str.maketrans(string.punctuation+string.ascii_letters, ' '*len(string.punctuation+string.ascii_letters))
    
# identify game archive and votecounter of interest
archive = '../data/archive.txt'
votecounter = VoteCounter

## Helper Function(s)

In [2]:
def _relevantGameInfo(game):
    
    # link
    link = game[:game.find('\n')]
    
    # thread number
    number = (link[link.find('&t=')+3:] if link.count('&')==1 
                      else link[link.find('&t=')+3:link.rfind('&')])

    # special events
    events, notes = {}, game[:game.find('\n\n')].split('\n')[-1]
    for note in notes.split('; '):
        if ' on post ' in note:
            postnumber = note.split(' on post ')[1]
            if postnumber in events:
                events[postnumber].append(note.split(' on post')[0])
            else:
                events[postnumber] = [note.split(' on post')[0]]
    
    # game title and number
    title = game.split('\n')[1]
    title_number = [i for i in title.translate(no_punctuation).split() if i.isdigit()][0]
    
    # moderator(s)
    moderators = game.split('\n')[2][len('Moderator: '):].split(', ')
    
    # living slots/players for each Day
    slots, players, fates, lynched = [], [], [], {}
    for line in game[game.find('\nPlayers\n')+9:].split('\n'):
        line = line.split(', ')
        
        # build list of players and slots
        players += line[0].split(' replaced ')
        slots.append(line[0].split(' replaced ')) 
        
        # extract last phase slot's vote helped decide Day
        # it's the phase they died, minus one if they were day-killed (not lynched)
        if 'survived' in line[-1].lower() or 'endgamed' in line[-1].lower():
            fates.append(float('inf'))
        else:
            phase = int(line[-1][line[-1].rfind(' ')+1:])
            #fate_modifier = 'killed day' in line[-1][:line[-1].rfind(' ')].lower() 
            fates.append(max(0, phase))
        
        # sort any detected lynches into `lynched` array
        if 'lynched' in line[-1].lower():
            lynched[phase] = slots[-1]

    return slots, players, fates, lynched, number, transitions[title_number], moderators, events

## VotecounterTest

In [9]:
#parameters
start_index = 0
end_index = 0

# open game archive, separate by game
with open(archive, encoding='utf-8') as f:
    games = f.read().split('\n\n\n')  

# open transitions archive, convert into dictionary of lists
with open('../data/transitions.tsv') as f:
    transitions = f.read()
transitions = transitions.split('\n')[3:]
transitions = {line.split('\t')[0]:line.strip().split('\t')[1:] for line in transitions if len(line.strip().split('\t')[1:]) > 0}

# process votes in each game's posts until a lynch found
# then store information about votecounter's performance
vote_results, vote_success, transition_results, transition_success, t0, total = {}, 0, {}, 0, time.time(), 0
end_index = end_index if end_index else len(games)  
for game_index, game in enumerate(games[start_index:end_index]):
    
    # temporarily only consider games where... (MODIFIABLE)
    if 'killed day 1' not in game.lower():
        continue
    
    # extract relevant information about this game
    slots, players, fates, lynched, number, game_transitions, moderators, events = _relevantGameInfo(game)
    with open('../data/posts/{}.jsonl'.format(number)) as f:
        gameposts =  [json.loads(l) for l in f]
        
    # prepare to collect data for this game
    transition_results[number] = []
    vote_results[number] = []
    
    #for day in range(1, len(game_transitions)):
    for day in [1]:
        
        # we'll remove this later; don't scan games that don't have this many games
        if len(game_transitions) < day+1:
            continue
    
        # identify day-specific information and set up voteextractor and votecount for them
        correct = lynched[day] if day in lynched else None
        start_point = 0 if day == 1 else int(game_transitions[day-2])
        relevant_slots = [slot for slot_index, slot in enumerate(slots) if fates[slot_index] >= day]
        relevant_players = []
        for slot in relevant_slots:
            relevant_players += slot
        votecount = VoteCount(relevant_slots, meta={'correct': correct})
        votecounter = VoteCounter.VoteExtracter(players=relevant_players)

        tphase, transition_start, transition_end, transition_match, transition_url = time.time(), None, None, False, None
        for post in gameposts[start_point:]:
            
            # prioritize any events associated with post
            if post['number'] in events:
                post_events = events[post['number']]
                for event in post_events:
                    sparevc = deepcopy(votecount)

                    # if event is a daykill, remove the player from votecount and voteextractor
                    if 'killed' == event.split(' ')[1]:
                        
                        # update relevant slots and players and make new votecounter
                        killed_player = event.split(' ')[0]
                        killed_slot = next(s for s in relevant_slots if s.count(killed_player) > 0)
                        del relevant_slots[relevant_slots.index(killed_slot)]
                        relevant_players = []
                        for slot in relevant_slots:
                            relevant_players += slot
                        votecounter = VoteCounter.VoteExtracter(players=relevant_players)
                        votecount.killplayer(killed_player, post['number'])

            # consider no more votes if voters have made a choice already
            elif not votecount.choice:

                # ignore posts not made by players
                if relevant_players.count(post['user']) == 0:
                    continue

                # update votecount for each vote found by votecounter
                # stop considering votes in post if votecount.choice
                for voted in votecounter.fromPost(post):
                    votecount.update(post['user'], voted, post['number'])
                    if votecount.choice:
                        vote_success += votecount.choice == correct
                        break

            # keep scanning to find newest post by game mod after detectedhammer
            elif not transition_start:
                if moderators.count(post['user']) > 0:
                    transition_start = int(post['number'])
                    transition_url = post['pagelink']

            # keep scanning to find last successive post by mod after they end Day
            elif not transition_end:
                if moderators.count(post['user']) == 0:
                    transition_end = int(post['number'])

                    # track match between inferred and transcribed transition post#
                    transition_match = int(game_transitions[day-1]) in list(range(transition_start, transition_end))
                    transition_success += transition_match

            # finish if votecount.choice, transition_start, and transition_end all populated
            else:
                break

        transition_results[number].append([list(range(transition_start, transition_end)), transition_url] if transition_start and transition_end else "None")
        vote_results[number].append(votecount)
        total += 1
        #if not (votecount.choice == correct and transition_match):
        print(day)
        print(game)
        #print(game.split('\n\n')[0])
        print(f'{game_index + start_index} {number} {vote_success} {transition_success} {total} {votecount.choice == correct} {transition_match} {time.time()-tphase}')
        print('\n---\n')

print(f'{vote_success/total} {transition_success/total} {total} {time.time()-t0}')

1
https://forum.mafiascum.net/viewtopic.php?f=53&t=15934
Game 1105: A Mafia Invasion!
Moderator: Zachrulez
Current Update: Town Win
Notes: AGar killed on post 283; Zdenek killed on post 490

Players
RobCapone, Mafia 2-Shot Rolecop, lynched Day 3
AGar, Town Mason, killed Day 1
Rain replaced Mute, Town Neighbor, survived
VP Baltar, Town Vanilla, killed Night 1
Empking replaced Guderian, Town 2-Shot Day Viglante, survived
Jerbs, Mafia Goon, lynched Day 1
LlamaGod, Town Vanilla, survived
Tasky, Town Vanilla, killed Night 2
GreyICE replaced RossWilliam, Town Vanilla, survived
Zdenek replaced Dizzle, Mafia Goon, killed Day 2
Locke Lamora replaced Vigilante Ventriloquist replaced Carrotcake, Town Neighbor, lynched Day 2
Ellibereth, Town Vanilla, survived
pappums rat, Town Vanilla, survived
5 15934 1 1 1 True True 0.14200139045715332

---

1
https://forum.mafiascum.net/viewtopic.php?f=53&t=16959
Game 1146: Don't Get Slapped in the Face by a Fish Mafia
Moderator: chesskid3
Current Update: Town 

In [4]:
display(HTML(post['content']))

In [8]:
current_votecount = sparevc.todict()
for each in current_votecount:
    if current_votecount[each]:
        print(each, '-', len(current_votecount[each]))
        for voter in current_votecount[each]:
            print(voter)
        print()

['Boonskiies', 'Aeronaut'] - 1
['Aquanim']

['oddmusic'] - 4
['eektor']
['pisskop']
['Boonskiies', 'Aeronaut']
['RadiantCowbells']

['RadiantCowbells'] - 2
['oddmusic']
['Metalcyanide']

['mykonian'] - 1
['Taly']

['Taly'] - 3
['Elyse']
['mykonian']
['Bellaphant', 'InsidiousLemons']

Not Voting - 2
['TheDudeAbides']
['toolenduso']



In [7]:
current_votecount = votecount.todict()
for each in current_votecount:
    if current_votecount[each]:
        print(each, '-', len(current_votecount[each]))
        for voter in current_votecount[each]:
            print(voter)
        print()

['Boonskiies', 'Aeronaut'] - 1
['Aquanim']

['RadiantCowbells'] - 1
['Metalcyanide']

['mykonian'] - 1
['Taly']

['Taly'] - 3
['Elyse']
['mykonian']
['Bellaphant', 'InsidiousLemons']

Not Voting - 6
['TheDudeAbides']
['toolenduso']
['Boonskiies', 'Aeronaut']
['eektor']
['RadiantCowbells']
['pisskop']



In [12]:
votecount.votesByVoted

[[7],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [8, 6],
 [11],
 [],
 [4, 9, 1],
 [3, 5, 0, 2, 8, 10],
 []]

In [25]:
sparevc.votesByVoted

[[7],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [2, 11, 0, 9],
 [8, 6],
 [12],
 [],
 [4, 10, 1],
 [3, 5],
 []]

In [24]:
sparevc.votesByVoter

[8, 12, 8, 13, 12, 13, 9, 0, 9, 8, 12, 8, 10]

In [22]:
sparevc.slots[8]

['oddmusic']

## Analyze Results
Will have to revise this to adapt to new variable names/codeflow.

In [4]:
# parameters
game_index = 177
postnumber = 0
day = 1
postnumber = str(postnumber)

# open game archive, separate by game
with open(archive, encoding='utf-8') as f:
    games = f.read().split('\n\n\n')  

# generate votecounter w/ specified players
game = games[game_index]
slots, players, fates, lynched, number, game_transitions, moderators, events = _relevantGameInfo(game)

# temporary DAY 2fix for until i run every day at once
#if vote_results[number][0]:
#    vote_results[number] = [None, None, vote_results[number][0]]
#    transition_results[number] = [None, None, transition_results[number][0]]

correct = lynched[day] if day in lynched else None
start_point = 0 if day == 1 else int(game_transitions[day-2])
if not int(postnumber):
    postnumber = game_transitions[day-1]
    
html('# {}'.format(number))
print(games[game_index])
print()
print('Vote Choice:', vote_results[number][day-1].choice)
print('Vote Correct:', vote_results[number][day-1].meta['correct'])

# relevant information and votecounter generation
relevant_slots = [slot for slot_index, slot in enumerate(slots) if fates[slot_index] >= day]
relevant_players = []
for slot in relevant_slots:
    relevant_players += slot
votecount = VoteCount(relevant_slots, meta={'correct': correct})
votecounter = VoteCounter.VoteExtracter(players=relevant_players)

# collect gameposts associated with game number
with open('../data/posts/{}.jsonl'.format(number)) as f:
    gameposts = [json.loads(l) for l in f]
    
# find and display selected post along with votecounter output for it
html('# Post {}'.format(postnumber))
post = next(item for item in gameposts if item["number"] == postnumber)
print('Extracted Votes:', list(votecounter.fromPost(post)))
print()
print(post)
print()
display(HTML(post['content']))
    
# display extracted and true phase transitions
html('# Phase Transitions')
print('True Transitions:', game_transitions)
print(transition_results[number][day-1])
html('[{}]({})'.format(*transition_results[number][day-1]))
transition_start, transition_end, transition_match, final_vote = None, None, False, None
for post in gameposts[start_point:int(postnumber)]:
    
    # prioritize any events associated with post
    if post['number'] in events:
        post_events = events[post['number']]
        for event in post_events:
            print(event)

            # if event is a daykill, remove the player from votecount and voteextractor
            if 'killed' == event.split(' ')[1]:

                # update relevant slots and players and make new votecounter
                killed_player = event.split(' ')[0]
                killed_slot = next(s for s in relevant_slots if s.count(killed_player) > 0)
                del relevant_slots[relevant_slots.index(killed_slot)]
                relevant_players = []
                for slot in relevant_slots:
                    relevant_players += slot
                votecounter = VoteCounter.VoteExtracter(players=relevant_players)
                votecount.killplayer(killed_player, post['number'])
                        
    # consider no more votes if voters have made a choice already
    elif not votecount.choice:

        # ignore posts not made by players
        if relevant_players.count(post['user']) == 0:
            continue

        # update votecount for each vote found by votecounter
        # stop considering votes in post if votecount.choice
        for voted in votecounter.fromPost(post):
            votecount.update(post['user'], voted, post['number'])
            if votecount.choice:
                print('Final Vote Post#', post['number'])
                break

    # keep scanning to find newest post by game mod after detectedhammer
    elif not transition_start:
        if moderators.count(post['user']) > 0:
            transition_start = int(post['number'])
            print('Transition Start:', transition_start)

    # keep scanning to find last successive post by mod after they end Day
    elif not transition_end:
        if moderators.count(post['user']) == 0:
            transition_end = int(post['number'])
            print('Transition End:', transition_start)

            # track match between inferred and transcribed transition post#
            transition_match = int(game_transitions[0]) in list(range(transition_start, transition_end))
            print('Transition Match:', transition_match)

    # finish if votecount.choice, transition_start, and transition_end all populated
    else:
        break
        
# display votecount up to indicated post number
html('# Current Votecount (Up to {})'.format(postnumber))
current_votecount = votecount.todict()
for each in current_votecount:
    if current_votecount[each]:
        print(each, '-', len(current_votecount[each]))
        for voter in current_votecount[each]:
            print(voter)
        print()

# display final votecount
html('# Final Votecount')
final_votecount = vote_results[number][day-1].todict()
for each in final_votecount:
    if final_votecount[each]:
        print(each, '-', len(final_votecount[each]))
        for voter in final_votecount[each]:
            print(voter)
        print()
        
# display final votelog
html('# Vote Log')
votelog = vote_results[number][day-1].votelog.copy()
for index, each in enumerate(votelog):
    each = each.split()
    each[-1] = '[{}]({})'.format(each[-1], next(item for item in gameposts if item["number"] == each[-1])['pagelink'])
    votelog[index] = ' '.join(each)
html('  \n'.join(reversed(votelog)))

https://forum.mafiascum.net/viewtopic.php?f=53&t=60920
Mini 1651: One Flew Over The Monkey's Nest
Moderator: MonkeyMan576
Current Update: Mafia Win
Notes: D1 no majority; oddmusic killed on post 501; Taly killed on post 864

Players
Boonskiies replaced Aeronaut, Vanilla Townie, endgamed day 3
Bellaphant replaced InsidiousLemons, Vanilla Townie, lynched day 3
eektor, Vanilla Townie, endgamed day 3
TheDudeAbides, Mafia Goon, survived
Elyse, Mafia Godfather, survived
toolenduso, Mafia Goon, survived
Metalcyanide, Vanilla Townie, endgamed day 3
Aquanim, Vanilla Townie, endgamed day 3
oddmusic, Vanilla Townie, killed day 1
RadiantCowbells, Town Cop, killed night 1
mykonian, Vanilla Townie, lynched day 2
pisskop, Town One-Shot Day Vigilante, killed night 2
Taly, Town Roleblocker, modkilled day 1



IndexError: list index out of range

In [38]:
relevant_slots

[['Boonskiies', 'Aeronaut'],
 ['Bellaphant', 'InsidiousLemons'],
 ['eektor'],
 ['TheDudeAbides'],
 ['Elyse'],
 ['toolenduso'],
 ['Metalcyanide'],
 ['Aquanim'],
 ['oddmusic'],
 ['RadiantCowbells'],
 ['mykonian'],
 ['pisskop'],
 ['Taly']]

In [27]:
for event in post_events:
    print(event)

["oddmusic"] killed


In [35]:
votecount.slots

[['Boonskiies', 'Aeronaut'],
 ['Bellaphant', 'InsidiousLemons'],
 ['eektor'],
 ['TheDudeAbides'],
 ['Elyse'],
 ['toolenduso'],
 ['Metalcyanide'],
 ['Aquanim'],
 ['RadiantCowbells'],
 ['mykonian'],
 ['pisskop'],
 ['Taly']]

In [29]:
votecount.killplayer(['Taly'])

StopIteration: 

In [39]:
html('# Current Votecount (Up to {})'.format(postnumber))
current_votecount = votecount.todict()
for each in current_votecount:
    if current_votecount[each]:
        print(each, '-', len(current_votecount[each]))
        for voter in current_votecount[each]:
            print(voter)
        print()

['Boonskiies', 'Aeronaut'] - 1
['Aquanim']

['oddmusic'] - 4
['eektor']
['pisskop']
['Boonskiies', 'Aeronaut']
['RadiantCowbells']

['RadiantCowbells'] - 2
['oddmusic']
['Metalcyanide']

['mykonian'] - 1
['Taly']

['Taly'] - 3
['Elyse']
['mykonian']
['Bellaphant', 'InsidiousLemons']

Not Voting - 2
['TheDudeAbides']
['toolenduso']

